### Load Modules and Data

In [ ]:
import pandas as pd
import pickle
import os
import numpy as np
import multiprocessing
import tmtoolkit

from sklearn.decomposition import LatentDirichletAllocation as LDA

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt

In [ ]:
rebuildModels = True
saveModels = True
buildViz = True
saveOutputs = True

In [ ]:
cores = multiprocessing.cpu_count()

In [ ]:
filePath = '../../data/processed/fromPython/' #path to load data files
files = os.listdir(filePath)
files = [x for x in files if '_mat.pkl' in x]
files = [x for x in files if '~' not in x] #in case files are open (windows)
files

['all_improve_learning_mat.pkl',
 'all_tech_challenges_mat.pkl',
 'family_improve_learning_mat.pkl',
 'family_tech_challenges_mat.pkl',
 'staff_improve_learning_mat.pkl',
 'staff_tech_challenges_mat.pkl',
 'student_improve_learning_mat.pkl',
 'student_tech_challenges_mat.pkl']

In [ ]:
matDict = {} #build directory 
for each in files:
    with open(filePath + each, 'rb') as f:
        matDict[each.split('.')[0]] = pickle.load(f)

In [ ]:
improveLearningAll = matDict['all_improve_learning_mat']
techChallengesAll = matDict['all_tech_challenges_mat']

In [ ]:
with open(filePath + 'all_tech_challenges_vectorizer.pkl', 'rb') as f:
    techChallengesVec = pickle.load(f)
    
with open(filePath + 'all_improve_learning_vectorizer.pkl', 'rb') as f:
    improveLearningVec = pickle.load(f)

In [ ]:
techChallengesAll.shape

(7527, 1206)

In [ ]:
improveLearningAll.shape

(8967, 1840)

### Analyzing Q1 Technology Challenges During Remote Learning





In [ ]:
topics = 8  #build a model with 8 topics for example
maxIter = 1000

In [ ]:
if rebuildModels:
    ldaTech = LDA(n_jobs = cores - 1, n_components = topics, random_state = 11, max_iter = maxIter)
    ldaTech.fit(techChallengesAll)
else:
    with open('../../data/models/fromPython/all_tech_challenges_lda_model.pkl', 'rb') as f:
        ldaTech = pickle.load(f)

In [ ]:
if saveModels:
    with open('../../data/models/fromPython/all_tech_challenges_lda_model.pkl', 'wb') as f:
        pickle.dump(ldaTech, f)

In [ ]:
wordWeightsTech = pd.DataFrame(ldaTech.components_, columns = techChallengesVec.get_feature_names()).transpose()

In [ ]:
wordWeightsTechNormed = wordWeightsTech.divide(wordWeightsTech.sum(axis = 1), axis = 0)

In [ ]:
for each in range(wordWeightsTech.shape[1]):
    current = wordWeightsTech[each]
    current = current.sort_values(ascending = False)
    current = current.index[:10]
    print(current)
#print the top 10 most frequent words of each topic: 
Index(['dont', 'ani', 'problem', 'use', 'know', 'challeng', 'like', 'thing',
       'note', 'power'],
      dtype='object')
Index(['meet', 'im', 'team', 'sometim', 'internet', 'kick', 'connect', 'class',
       'let', 'teacher'],
      dtype='object')
Index(['time', 'screen', 'class', 'challeng', 'day', 'dure', 'hard', 'kid',
       'learn', 'mani'],
      dtype='object')
Index(['comput', 'laptop', 'school', 'slow', 'work', 'use', 'wifi', 'sometim',
       'veri', 'restart'],
      dtype='object')
Index(['ipad', 'issu', 'work', 'hi', 'video', 'youtub', 'use', 'seesaw', 'sp',
       'distract'],
      dtype='object')
Index(['student', 'need', 'use', 'learn', 'tech', 'teacher', 'school',
       'support', 'help', 'thi'],
      dtype='object')
Index(['team', 'work', 'chat', 'microsoft', 'use', 'zoom', 'sometim', 'camera',
       'screen', 'room'],
      dtype='object')
Index(['teacher', 'assign', 'schoolog', 'work', 'class', 'ha', 'link', 'time',
       'differ', 'submit'],
      dtype='object')

In [ ]:
for each in range(wordWeightsTechNormed.shape[1]):
    current = wordWeightsTechNormed[each]
    current = current.sort_values(ascending = False)
    current = current.index[:10]
    print(current)
Index(['dont', 'ani', 'power', 'stuff', 'nope', 'outag', 'test', 'touch', 'la',
       'attent'],
      dtype='object')
Index(['meet', 'im', 'kick', 'wait', 'join', 'drop', 'echo', 'lobbi',
       'occasion', 'boot'],
      dtype='object')
Index(['small', 'type', 'young', 'file', 'eye', 'relat', 'impact', 'adult',
       'japanes', 'listen'],
      dtype='object')
Index(['comput', 'slow', 'wifi', 'laggi', 'batteri', 'shut', 'somtim', 'swap',
       'charger', 'jack'],
      dtype='object')
Index(['ipad', 'youtub', 'mic', 'block', 'upload', 'password', 'print',
       'tablet', 'account', 'weve'],
      dtype='object')
Index(['support', 'famili', 'educ', 'staff', 'train', 'best', 'actual',
       'appropri', 'assist', 'themselv'],
      dtype='object')
Index(['chat', 'microsoft', 'zoom', 'suck', 'whiteboard', 'outlook', 'green',
       'black', 'version', 'featur'],
      dtype='object')
Index(['assign', 'submit', 'complet', 'send', 'post', 'schedul', 'sourc',
       'mark', 'calendar', 'window'],
      dtype='object')

In [ ]:
if buildViz:
    viz = pyLDAvis.sklearn.prepare(ldaTech, techChallengesAll, techChallengesVec)
    pyLDAvis.save_html(viz, '../../data/models/fromPython/all_tech_challenges_lda_viz.html')
    #to build a visualization of how each survey response is located onto each topic.

In [ ]:
techChallengesAllLda = ldaTech.transform(techChallengesAll)
techChallengesStudentsLda = ldaTech.transform(matDict['student_tech_challenges_mat'])
techChallengesStaffLda = ldaTech.transform(matDict['staff_tech_challenges_mat'])
techChallengesFamilyLda = ldaTech.transform(matDict['family_tech_challenges_mat'])

In [ ]:
ldaTech

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=1000,
                          mean_change_tol=0.001, n_components=8, n_jobs=15,
                          perp_tol=0.1, random_state=11, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
if saveOutputs:
    with open('../../data/models/fromPython/all_tech_challenges_topics.pkl', 'wb') as f:
        pickle.dump(techChallengesAllLda, f)
    
    with open('../../data/models/fromPython/student_tech_challenges_topics.pkl', 'wb') as f:
        pickle.dump(techChallengesStudentsLda, f)
        
    with open('../../data/models/fromPython/staff_tech_challenges_topics.pkl', 'wb') as f:
        pickle.dump(techChallengesStaffLda, f)
        
    with open('../../data/models/fromPython/family_tech_challenges_topics.pkl', 'wb') as f:
        pickle.dump(techChallengesFamilyLda, f)

### Analyzing Q2 Improve Learning

-Conduct the same topic modeling for a survey question #2 about how family, staff and students suggested ways to improve remote learning experiences



In [ ]:
topics = 8

In [ ]:
if rebuildModels:
    ldaImprove = LDA(n_jobs = cores - 1, n_components = topics, random_state = 5, max_iter = maxIter)
    ldaImprove.fit(improveLearningAll)
else:
    with open('../../data/models/fromPython/all_improve_learning_lda_model.pkl', 'rb') as f:
        ldaTech = pickle.load(f)

In [ ]:
if saveModels:
    with open('../../data/models/fromPython/all_improve_learning_lda_model.pkl', 'wb') as f:
        pickle.dump(ldaImprove, f)

In [ ]:
if buildViz:
    viz = pyLDAvis.sklearn.prepare(ldaImprove, improveLearningAll, improveLearningVec)
    pyLDAvis.save_html(viz, '../../data/models/fromPython/all_improve_learning_lda_viz.html')

In [ ]:
improveLearningAllLda = ldaImprove.transform(improveLearningAll)
improveLearningStudentsLda = ldaImprove.transform(matDict['student_improve_learning_mat'])
improveLearningStaffLda = ldaImprove.transform(matDict['staff_improve_learning_mat'])
improveLearningFamilyLda = ldaImprove.transform(matDict['family_improve_learning_mat'])

In [ ]:
if saveOutputs:
    with open('../../data/models/fromPython/all_improve_learning_topics.pkl', 'wb') as f:
        pickle.dump(improveLearningAllLda, f)
    
    with open('../../data/models/fromPython/student_improve_learning_topics.pkl', 'wb') as f:
        pickle.dump(improveLearningStudentsLda, f)
        
    with open('../../data/models/fromPython/staff_improve_learning_topics.pkl', 'wb') as f:
        pickle.dump(improveLearningStaffLda, f)
        
    with open('../../data/models/fromPython/family_improve_learning_topics.pkl', 'wb') as f:
        pickle.dump(improveLearningFamilyLda, f)